<a href="https://colab.research.google.com/github/Gonzy/tests1/blob/master/ResumableLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install ultralytics --upgrade -q > /dev/null

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
!ls /content/gdrive/MyDrive/datasets/

Mounted at /content/gdrive
project_1  test.zip


In [3]:
!mkdir -p /content/merged
!cp /content/gdrive/MyDrive/datasets/project_1/merged.tar.gz /content/merged.tar.gz
!tar -xzvf merged.tar.gz -C /content/merged > /dev/null

In [ ]:
#!cp /content/gdrive/MyDrive/datasets/test.zip /content/test.zip
#!unzip test.zip -d /content/test_dataset > /dev/null

In [ ]:
# !rm -rf /content/gdrive/MyDrive/datasets/checkpoints/project_1/
# !mkdir /content/gdrive/MyDrive/datasets/project_1/checkpoints/

In [4]:
!mkdir -p /content/runs/detect/scratch-detector
%cd /content/runs/detect/scratch-detector
!tar -xzvf /content/gdrive/MyDrive/datasets/project_1/checkpoints/last.tar.gz > /dev/null

/content/runs/detect/scratch-detector


In [ ]:
%cd /content/

from ultralytics import YOLO
import os
import shutil
from pathlib import Path
os.environ["TORCH_ENABLE_DYNAMO"] = "0"
os.environ["_TORCH_DYNAMO_DISABLE"] = "1"
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

checkpoint_dir = "/content/gdrive/MyDrive/datasets/project_1/checkpoints"
last_filename = "last.tar.gz"
last_gz_path = f"{checkpoint_dir}/{last_filename}"

def on_fit_epoch_end(trainer):

    run_dir = Path("/content/") / trainer.save_dir.resolve()
    print("start archiving...")
    !tar -czvf {last_filename} -C {run_dir} . > /dev/null
    print("end archiving...")

    if trainer.epoch > 0 and os.path.exists(last_gz_path):
      print("Rename prev file")
      !mv "{last_gz_path}" "{checkpoint_dir}/epoch/epoch{trainer.epoch + 1}.tar.gz"

    !cp {last_filename} "{checkpoint_dir}"
    !rm {last_filename}
    print("backup complete!")


is_resume = False
model = "/content/runs/detect/scratch-detector/weights/last.pt"

if os.path.exists(model):
  is_resume = True
  print("Resume training")
else:
  model = 'yolo11m.pt'
  print("Start training")

model = YOLO(model)
model.add_callback('on_fit_epoch_end', on_fit_epoch_end)

results = model.train(
    data='/content/merged/data.yaml',
    epochs=200,
    imgsz=640,
    batch=16,
    name="scratch-detector",
    device=0,
    val=True,
    save=True,
    cache=False,
    save_dir="/content/runs/",
    resume=is_resume,
    lr0=0.001,
    lrf=0.05,
    cos_lr=True,
    warmup_epochs=4,
    warmup_momentum=0.8,
    warmup_bias_lr=0.1,
    patience=15,
    mosaic=0.0,
    mixup=0.0,
    hsv_h=0.0,
    hsv_s=0.0,
    hsv_v=0.0,
    fliplr=0.1,
    flipud=0.0,
    translate=0.05,
    scale=0.05,
    perspective=0.0,
    freeze=0
)

/content
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Resume training
Ultralytics 8.3.156 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=/content/merged/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=200, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, ker

100%|██████████| 755k/755k [00:00<00:00, 143MB/s]



                   from  n    params  module                                       arguments                     
  0                  -1  1      1856  ultralytics.nn.modules.conv.Conv             [3, 64, 3, 2]                 
  1                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  2                  -1  1    111872  ultralytics.nn.modules.block.C3k2            [128, 256, 1, True, 0.25]     
  3                  -1  1    590336  ultralytics.nn.modules.conv.Conv             [256, 256, 3, 2]              
  4                  -1  1    444928  ultralytics.nn.modules.block.C3k2            [256, 512, 1, True, 0.25]     
  5                  -1  1   2360320  ultralytics.nn.modules.conv.Conv             [512, 512, 3, 2]              
  6                  -1  1   1380352  ultralytics.nn.modules.block.C3k2            [512, 512, 1, True]           
  7                  -1  1   2360320  ultralytics.nn.modules.conv.Conv             [512

100%|██████████| 5.35M/5.35M [00:00<00:00, 322MB/s]


AMP: checks passed ✅
train: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1809.3±986.8 MB/s, size: 140.3 KB)


...

train: /content/merged/train/images/10231280-honda-honda_wr-v_i-vtec_vx-2020-minor_scratch-dents-kyzoprhnsn6t7wzygbbsew_jpeg.rf.727adb4bddc20c77b94e2b6e341c660c.jpg: 1 duplicate labels removed
train: /content/merged/train/images/10273265-mahindra-mahindra_tuv300_2015-2020_t10-2018-minor_scratch-dents-8qeegk1otmwprqb2qodwta_jpeg.rf.4ce5a9f8fc822aa8911fa89e34efcb38.jpg: 1 duplicate labels removed
train: /content/merged/train/images/8368960-hyundai-hyundai_elite_i20_2014-2018_asta_12-2014-minor_scratch-dents-a8mnu2soqlweeejh-2bsgcla_jpeg.rf.482e12f708419970542a1bd2367a7422.jpg: 1 duplicate labels removed
train: /content/merged/train/images/8717563-hyundai-hyundai_grand_i10_nios_asta_amt_12_kappa_vtvt-2021-minor_scratch-dents-og5xnheuqvsdx4qu3sasta_jpeg.rf.a0b6741977d96ee551f56c006069af7a.jpg: 1 duplicate labels removed
train: /content/merged/train/images/9271257-renault-renault_scala_2012-2018_rxl-2016-minor_scratch-dents-lunybi9brumj9nfmzzq_ta_jpeg.rf.45f2847db726cbf65174b53bfbe341ce.jpg

train: New cache created: /content/merged/train/labels.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 16.6±9.5 MB/s, size: 61.8 KB)


val: Scanning /content/merged/valid/labels... 4862 images, 21 backgrounds, 0 corrupt: 100%|██████████| 4862/4862 [00:11<00:00, 441.69it/s]


val: New cache created: /content/merged/valid/labels.cache
Plotting labels to runs/detect/scratch-detector/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.005' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: SGD(lr=0.01, momentum=0.9) with parameter groups 106 weight(decay=0.0), 113 weight(decay=0.0005), 112 bias(decay=0.0)
Resuming training /content/runs/detect/scratch-detector/weights/last.pt from epoch 30 to 200 total epochs
Image sizes 640 train, 640 val
Using 2 dataloader workers
Logging results to runs/detect/scratch-detector
Starting training for 200 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     30/200      7.52G      1.295      1.686      1.501         53        640:  53%|█████▎    | 746/1419 [07:27<06:35,  1.70it/s]

In [5]:
#!rm -rf /content/content
# !rm -rf /content/merged/
!rm -rf /content/runs
#!rm -rf /content/gdrive/MyDrive/models/checkpoints/test/
#!mkdir -P /content/gdrive/MyDrive/models/checkpoints/test/
#!ls /content/gdrive/MyDrive/models/checkpoints/test/
